In [ ]:

# Importação das Bibliotecas

import requests
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine
from datetime import datetime

# Opções pandas
pd.set_option("display.max_columns", None)

# Configurações da conexão com o Postgres 
DB_USER = "usuário do banco de dados"
DB_PASS = "senha do banco de dados"
DB_HOST = "localhost"
DB_NAME = "nome do bando de dados"

def get_engine(user=DB_USER, pwd=DB_PASS, host=DB_HOST, db=DB_NAME):
    return create_engine(f"postgresql+psycopg2://{user}:{pwd}@{host}/{db}")
    
  
# Coleta a tabela de resultado do site Fundamentus e retorna o Dataframe bruto   
def get_fundamentus_data():
        
        
    url = "http://www.fundamentus.com.br/resultado.php"
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0 Safari/537.36")
    }
    resp = requests.get(url, headers=headers, timeout=30)
    resp.encoding = "latin1"

    tables = pd.read_html(resp.text, decimal=",", thousands=".")
    if not tables:
        raise ValueError("Nenhuma tabela encontrada no HTML do Fundamentus.")
    return tables[0] 

# testando a saída do Dataframe 
df_raw_fund = get_fundamentus_data()
df_raw_fund.head()



# Padronização dos Dados
def clean_fundamentus_data(df):
    
    
    df = df.copy()
    df.columns = [c.strip().lower().replace(" ", "_").replace("/", "_") for c in df.columns]

    # Garantir existência da coluna de ticker (costuma ser 'papel')
    if "papel" in df.columns:
        df.rename(columns={"papel": "ticker"}, inplace=True)

    # Converter numéricos (ignora a primeira coluna assumida como ticker)
    for col in df.columns:
        if col == "ticker":
            continue
        try:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace(".", "", regex=False)
                .str.replace(",", ".", regex=False)
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")
        except Exception:
            pass

    # Ticker raiz (sem .SA) — para relacionar com Yahoo
    if "ticker" in df.columns:
        df["ticker_root"] = df["ticker"].astype(str).str.upper().str.strip()
    else:
        raise ValueError("Coluna de ticker não encontrada no Fundamentus.")

    # Timestamp de carga (bom pra auditoria)
    df["dt_carga"] = pd.Timestamp.utcnow()

    return df

# Testando a saida do DataFrame 
df_fund = clean_fundamentus_data(df_raw_fund)
df_fund.head()


def save_to_postgres(df, tabela, if_exists="replace"):
    eng = get_engine()
    df.to_sql(tabela, eng, if_exists=if_exists, index=False)
    print(f" Dados salvos na tabela '{tabela}' ({len(df)} linhas).")
    
    
def run_pipeline_fundamentus():
    print(" Fundamentus: iniciando...")
    df_raw = get_fundamentus_data()
    print(" Coleta ok.")
    df_clean = clean_fundamentus_data(df_raw)
    print(" Limpeza ok.")
    save_to_postgres(df_clean, "acoes_info", if_exists="replace")
    print(" Fundamentus: concluído.")
    return df_clean

# Execução do pipeline Fundamentus
df_fund_final = run_pipeline_fundamentus()
df_fund_final.head()


# Coleta do histórico de preços do Yahoo Finance para uma lista de tickers .SA.
# Retorna DataFrame consolidado e com colunas achatadas.

def get_yahoo_data(tickers, start="2020-01-01", end=None):
        
    
    if end is None:
        end = datetime.utcnow().strftime("%Y-%m-%d")

    frames = []
    for t in tickers:
        try:
            df = yf.download(t, start=start, end=end, progress=False)
            if df.empty:
                print(f" Sem dados para {t}.")
                continue
            df["ticker"] = t
            frames.append(df)
            print(f" Coletado {t} ({len(df)} linhas).")
        except Exception as e:
            print(f" Erro em {t}: {e}")

    if not frames:
        raise ValueError("Nenhum dado coletado do Yahoo Finance.")

    out = pd.concat(frames).reset_index()

    # Ajuste para evitar o erro das tuplas
    if isinstance(out.columns, pd.MultiIndex):
        out.columns = ["_".join([str(c) for c in col if c]) for col in out.columns]

    return out

# Testando a coleta feita no app Yahoo Finance
tickers = ["PETR4.SA", "VALE3.SA", "ITUB4.SA"]
df_yf_raw = get_yahoo_data(tickers)
df_yf_raw.head()



 # Padroniza nomes e tipos; cria ticker_root para relacionamento.
def clean_yahoo_data(df):
        
   
    df = df.copy()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]

    # Garantir tipos
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

    num_cols = ["open", "high", "low", "close", "adj_close", "volume"]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # ticker_root: remove .SA
    if "ticker" in df.columns:
        df["ticker_root"] = df["ticker"].astype(str).str.upper().str.replace(".SA", "", regex=False)
    else:
        raise ValueError("Coluna 'ticker' ausente no Yahoo.")

    df["dt_carga"] = pd.Timestamp.utcnow()
    return df


# Função para testar a coleta do Yahoo Finance 
def run_pipeline_yahoo(tickers=None, start="2020-01-01", end=None):
    if tickers is None:
        
        # tickers .SA precisam existir — aqui vou usar 3 clássicos
        tickers = ["PETR4.SA", "VALE3.SA", "ITUB4.SA"]

    print(" Yahoo: iniciando...")
    df_raw = get_yahoo_data(tickers, start=start, end=end)
    print(" Coleta ok.")
    df_clean = clean_yahoo_data(df_raw)
    print(" Limpeza ok.")
    save_to_postgres(df_clean, "acoes_historico", if_exists="replace")
    print(" Yahoo: concluído.")
    return df_clean

# Execução da função
df_yf_final = run_pipeline_yahoo()
df_yf_final.head()


eng = get_engine()

sql_check_1 = "SELECT ticker, ticker_root, count(*) AS n FROM acoes_info GROUP BY 1,2 ORDER BY n DESC LIMIT 5;"
sql_check_2 = "SELECT ticker, ticker_root, MIN(date) AS dt_min, MAX(date) AS dt_max, COUNT(*) AS n FROM acoes_historico GROUP BY 1,2 ORDER BY n DESC LIMIT 5;"

df_chk1 = pd.read_sql(sql_check_1, eng)
df_chk2 = pd.read_sql(sql_check_2, eng)

df_chk1, df_chk2

C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text, decimal=",", thousands=".")


 Fundamentus: iniciando...


C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text, decimal=",", thousands=".")


 Coleta ok.
 Limpeza ok.
 Dados salvos na tabela 'acoes_info' (994 linhas).
 Fundamentus: concluído.


C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:116: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow().strftime("%Y-%m-%d")
C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:121: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)


...

C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:121: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)


 Coletado ITUB4.SA (1426 linhas).
 Yahoo: iniciando...
 Coletado PETR4.SA (1426 linhas).
 Coletado VALE3.SA (1426 linhas).
 Coletado ITUB4.SA (1426 linhas).
 Coleta ok.
 Limpeza ok.


C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:116: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = datetime.utcnow().strftime("%Y-%m-%d")
C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:121: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)
C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:121: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)
C:\Users\felyp\AppData\Local\Temp\ipykernel_3240\952577549.py:121: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(t, start=start, end=end, progress=False)


 Dados salvos na tabela 'acoes_historico' (4278 linhas).
 Yahoo: concluído.


(   ticker ticker_root  n
 0  MRSA5B      MRSA5B  2
 1  MRSA3B      MRSA3B  2
 2  MRSA6B      MRSA6B  2
 3  AGEN33      AGEN33  1
 4   ALOS3       ALOS3  1,
      ticker ticker_root     dt_min     dt_max     n
 0  PETR4.SA       PETR4 2020-01-02 2025-09-22  1426
 1  ITUB4.SA       ITUB4 2020-01-02 2025-09-22  1426
 2  VALE3.SA       VALE3 2020-01-02 2025-09-22  1426)